In [ ]:
#|default_exp merge

# merge

> Fix merge conflicts in jupyter notebooks

In [ ]:
#|export
from nbprocess.imports import *
from nbprocess.read import *
from nbprocess.export import *
from nbprocess.sync import *
from fastcore.script import *

from difflib import SequenceMatcher
import json

## Introduction

When working with jupyter notebooks (which are json files behind the scenes) and GitHub, it is very common that a merge conflict (that will add new lines in the notebook source file) will break some notebooks you are working on. This module defines the function `fix_conflicts` to fix those notebooks for you, and attempt to automatically merge standard conflicts. The remaining ones will be delimited by markdown cells like this:

`<<<<<< HEAD`

In [ ]:
# local code here

`======`

In [ ]:
# remote code here

`>>>>>> a7ec1b0bfb8e23b05fd0a2e6cafcb41cd0fb1c35`

Below is an example of broken notebook. The json format is broken by the lines automatically added by git. Such a file can't be opened in jupyter notebook.

In [ ]:
broken = Path('../tests/example.ipynb.broken')
tst_nb = broken.read_text()
print(tst_nb)

{
 "cells": [
  {
   "cell_type": "code",
   "execution_count": 6,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "3"
      ]
     },
     "execution_count": 6,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
<<<<<<< HEAD
    "z=3\n",
    "z=2\n",
>>>>>>> a7ec1b0bfb8e23b05fd0a2e6cafcb41cd0fb1c35
    "z"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 7,
   "execution_count": 5,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "6"
      ]
     },
<<<<<<< HEAD
     "execution_count": 7,
     "execution_count": 5,
>>>>>>> a7ec1b0bfb8e23b05fd0a2e6cafcb41cd0fb1c35
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "x=3\n",
    "y=3\n",
    "x+y"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": []
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",

Note that in this example, the second conflict is easily solved: it just concerns the execution count of the second cell and can be solved by choosing either option without really impacting your notebook. This is the kind of conflict we will fix automatically. The first conflict is more complicated as it spans across two cells and there is a cell present in one version, not the other. Such a conflict (and generally the ones where the inputs of the cells change form one version to the other) aren't automatically fixed, but we will return a proper json file where the annotations introduced by git will be placed in markdown cells.

## Creating a merged notebook

The approach we use is to first "unpatch" the conflicted file, regenerating the two files it was originally created from. Then we redo the diff process, but using cells instead of text lines.

In [ ]:
#|export
_BEG,_MID,_END = '<'*7,'='*7,'>'*7
conf_re = re.compile(rf'^{_BEG}\s+(\S+)\n(.*?)\n{_MID}\n(.*?)^{_END}\s+(\S+)\n', re.MULTILINE|re.DOTALL)

def _unpatch_f(before, cb1, cb2, c, r):
    if cb1 is not None and cb1 != cb2: raise Exception(f'Branch mismatch: {cb1}/{cb2}')
    r.append(before)
    r.append(c)
    return cb2

In [ ]:
#|export
def unpatch(s:str):
    "Takes a string with conflict markers and returns the two original files, and their branch names"
    *main,last = conf_re.split(s)
    r1,r2,c1b,c2b = [],[],None,None
    for before,c1_branch,c1,c2,c2_branch in chunked(main, 5):
        c1b = _unpatch_f(before, c1b, c1_branch, c1, r1)
        c2b = _unpatch_f(before, c2b, c2_branch, c2, r2)
    return ''.join(r1+[last]), ''.join(r2+[last]), c1b, c2b

The result of "unpatching" our conflicted test notebook is the two original notebooks it would have been created from. Each of these original notebooks will contain valid JSON:

In [ ]:
a,b,branch1,branch2 = unpatch(tst_nb)
json.loads(a)

{'cells': [{'cell_type': 'code',
   'execution_count': 6,
   'metadata': {},
   'outputs': [{'data': {'text/plain': ['3']},
     'execution_count': 6,
     'metadata': {},
     'output_type': 'execute_result'}],
   'source': ['z=3\n', 'z']},
  {'cell_type': 'code',
   'execution_count': 5,
   'metadata': {},
   'outputs': [{'data': {'text/plain': ['6']},
     'execution_count': 7,
     'metadata': {},
     'output_type': 'execute_result'}],
   'source': ['x=3\n', 'y=3\n', 'x+y']},
  {'cell_type': 'code',
   'execution_count': None,
   'metadata': {},
   'outputs': [],
   'source': []}],
 'metadata': {'kernelspec': {'display_name': 'Python 3',
   'language': 'python',
   'name': 'python3'}},
 'nbformat': 4,
 'nbformat_minor': 2}

In [ ]:
json.loads(b)

{'cells': [{'cell_type': 'code',
   'execution_count': 6,
   'metadata': {},
   'outputs': [{'data': {'text/plain': ['3']},
     'execution_count': 6,
     'metadata': {},
     'output_type': 'execute_result'}],
   'source': ['z=2\n', 'z']},
  {'cell_type': 'code',
   'execution_count': 5,
   'metadata': {},
   'outputs': [{'data': {'text/plain': ['6']},
     'execution_count': 5,
     'metadata': {},
     'output_type': 'execute_result'}],
   'source': ['x=3\n', 'y=3\n', 'x+y']},
  {'cell_type': 'code',
   'execution_count': None,
   'metadata': {},
   'outputs': [],
   'source': []}],
 'metadata': {'kernelspec': {'display_name': 'Python 3',
   'language': 'python',
   'name': 'python3'}},
 'nbformat': 4,
 'nbformat_minor': 2}

In [ ]:
branch1,branch2

('HEAD', 'a7ec1b0bfb8e23b05fd0a2e6cafcb41cd0fb1c35')

In [ ]:
#|export
def _make_md(code): return [dict(source=f'`{code}`', cell_type="markdown", metadata={})]
def _make_conflict(a,b, branch1, branch2):
    return _make_md(f'{_BEG} {branch1}') + a+_make_md(_MID)+b + _make_md(f'{_END} {branch2}')

def _merge_cells(a, b, brancha, branchb, theirs):
    matches = SequenceMatcher(None, a, b).get_matching_blocks()
    res,prev_sa,prev_sb,conflict = [],0,0,False
    for sa,sb,sz in matches:
        ca,cb = a[prev_sa:sa],b[prev_sb:sb]
        if ca or cb:
            res += _make_conflict(ca, cb, brancha, branchb)
            conflict = True
        if sz: res += b[sb:sb+sz] if theirs else a[sa:sa+sz]
        prev_sa,prev_sb = sa+sz,sb+sz
    return res,conflict

In [ ]:
#|export
@call_parse
def fix_merge(nbname:str, # notebook filename to fix
              outname:str=None, # filename of output notebook, defaults to `nbname`
              nobackup:bool=True, # do not backup `nbname` to `nbname.bak` if `outname` not provided
              theirs:bool=False, # use their outputs/metadata instead of ours
              noprint:bool=False): # Do not print info about whether conflict found
    "Create working notebook from conflicted notebook `nbname`"
    nbname = Path(nbname)
    if not nobackup and not outname: shutil.copy(fname, fname.with_suffix('.ipynb.bak'))
    nbtxt = nbname.read_text()
    a,b,branch1,branch2 = unpatch(nbtxt)
    ac,bc = dict2nb(json.loads(a)),dict2nb(json.loads(b))
    dest = bc if theirs else ac
    cells,conflict = _merge_cells(ac.cells, bc.cells, branch1, branch2, theirs=theirs)
    dest.cells = cells
    write_nb(dest, ifnone(outname, nbname))
    if not noprint:
        if conflict: print("One or more conflict remains in the notebook, please inspect manually.")
        else: print("Successfully merged conflicts!")
    return conflict

This begins by optionally backing the notebook `fname` to `fname.bak` in case something goes wrong. Then it parses the broken json, solving conflicts in cells. Every conflict that only involves metadata or outputs of cells will be solved automatically by using the local (`theirs==False`) or the remote (`theirs==True`) branch. Otherwise, or for conflicts involving the inputs of cells, the json will be repaired by including the two version of the conflicted cell(s) with markdown cells indicating the conflicts. You will be able to open the notebook again and search for the conflicts (look for `<<<<<<<`) then fix them as you wish.

A message will be printed indicating whether the notebook was fully merged or if conflicts remain.

In [ ]:
fix_merge(broken, outname='tmp.ipynb')
chk = read_nb('tmp.ipynb')
test_eq(len(chk.cells), 7)

One or more conflict remains in the notebook, please inspect manually.


## Export-

In [ ]:
#|hide
from nbprocess.doclinks import nbs_export
nbs_export()